## Convert *existing* paradigm (PsychToolbox input) into FreeSurfer-friendly .par files

In [2]:
import os
import pandas as pd
import numpy as np

behav_dir='/autofs/space/lilli_001/users/DARPA-newARC/behavior/params'
par_path='/autofs/space/lilli_001/users/DARPA-newARC/behavior/params/optseq'

df = pd.read_csv(os.path.join(behav_dir,'mri_newarc.csv'),index_col=False)
df=pd.DataFrame(df)

ConflictType=[]
ITI=[]
Reward=[]
Risk=[]
Trial=[]


for i in df.ConflictType:
    ConflictType.append('NULL')
    ConflictType.append(i)
    
for i in df.ITI:
    ITI.append(i) 
    ITI.append(5.25)

Time=[0]
s=0

for i in ITI:
    s=s+i
    Time.append(s)

Time=Time[:-1]
    
df2_data={'ConflictType':ConflictType,'ITI':ITI,'Time':Time}
    
df2=pd.DataFrame(data=df2_data)
df2['Time']=Time

trial_1 = df2.iloc[0:162]
trial_2 = df2.iloc[162:324]
trial_3 = df2.iloc[324::]

trial_1.to_csv(os.path.join(par_path,'3cond_mod-001.csv'))
trial_2.to_csv(os.path.join(par_path,'3cond_mod-002.csv'))
trial_3.to_csv(os.path.join(par_path,'3cond_mod-003.csv'))
df2.to_csv(os.path.join(par_path,'3cond_mod-AllRuns.csv'))

## Concatenate Existing files

In [11]:
import os
import pandas as pd
import numpy as np

bdir='/autofs/space/lilli_001/users/DARPA-newARC/behavior'

subjects = ['newARC_001','newARC_004','newARC_005','newARC_008',\
            'newARC_009','newARC_010','newARC_011','newARC_014',\
            'newARC_015','newARC_016','newARC_017','newARC_018',\
            'newARC_020']

for subject in subjects:
    df=pd.DataFrame()
    try:
        df1=pd.read_csv(os.path.join(bdir,'%s_run_1.csv' % subject),index_col=False)
        df1=pd.DataFrame(df1)
        df.append(df1)
    except IOError:
        print '%s does not have run 1' % subject
        continue
    try:
        df2=pd.read_csv(os.path.join(bdir,'%s_run_2.csv' % subject),index_col=False)
        df2=pd.DataFrame(df2)
        df.append(df2)
    except IOError:
        print '%s does not have run 2' % subject
        continue
    try:
        df3=pd.read_csv(os.path.join(bdir,'%s_run_3.csv' % subject),index_col=False)
        df3=pd.DataFrame(df3)
        df.append(df3)
    except IOError:
        print '%s does not have run 1' % subject
        continue
    display(df1)    
        
#     if df1 and df2 and df3 == True:
#         df = pd.concat([df1, df2, df3], ignore_index=True)
#     elif:
#         '%s does not have all three runs'

#     df.to_csv(os.path.join(par_path,'%s_allruns.csv'% subejct))

,ConflictType,ITI,Reward,Risk,Trial,TimeType,RiskType,RewardType,Shock,LeftKey,FixOnset,RiskOnset,RewardOnset,ResponseType,ResponseOnset,ResponseTime,ResponseUncertainty,ResultOnset,ShockOnset
0,medconflict,1.750,7.32,0.19,1,1,1,1,0,0,0.024643,1.777002,2.292563,1,3.863141,1.570578,0.000293,5.781429,NaN
1,highconflict,1.750,13.90,0.85,2,1,3,3,1,0,6.777979,8.547789,9.047388,0,9.703334,0.655946,0.000502,12.530900,NaN
2,highconflict,2.625,9.76,0.79,3,1,3,2,1,0,13.531360,16.167128,16.682949,0,17.894698,1.211749,0.000284,20.167909,NaN
3,medconflict,2.625,5.37,0.44,4,1,2,1,1,0,21.168377,23.795017,24.310444,1,26.398583,2.088139,0.000280,27.792931,28.269899
4,highconflict,1.750,7.32,0.62,5,1,2,1,0,1,28.792880,30.543452,31.058857,1,32.951388,1.892531,0.001166,34.541391,NaN
5,lowconflict,2.625,11.71,0.45,6,1,2,3,1,0,35.542010,38.177808,38.693576,1,39.542137,0.848561,0.000254,42.178442,42.654220
6,lowconflict,1.750,11.46,0.22,7,1,1,2,1,0,43.179022,44.940102,45.455475,1,46.166026,0.710550,0.000276,48.937959,49.410126
7,medconflict,5.250,14.75,0.74,8,1,3,3,1,0,49.932182,55.205333,55.703079,1,57.541691,1.838613,0.000280,59.186784,59.663362
8,medconflict,2.625,13.29,0.66,9,1,3,3,0,0,60.187151,62.835935,63.334594,1,64.621376,1.286783,0.000262,66.823831,NaN
9,medconflict,2.625,5.85,0.55,10,1,2,1,0,0,67.816822,70.451386,70.949364,0,71.973330,1.023966,0.000350,74.448607,NaN


newARC_004 does not have run 1


,ConflictType,ITI,Reward,Risk,Trial,TimeType,RiskType,RewardType,Shock,LeftKey,FixOnset,RiskOnset,RewardOnset,ResponseType,ResponseOnset,ResponseTime,ResponseUncertainty,ResultOnset,ShockOnset
0,medconflict,1.750,7.32,0.19,1,1,1,1,0,1,0.124341,1.856485,2.418241,1,3.319280,0.901040,0.000419,5.791193,NaN
1,highconflict,1.750,13.90,0.85,2,1,3,3,0,1,6.790634,8.549954,9.064476,1,10.335121,1.270645,0.000756,12.546804,NaN
2,highconflict,2.625,9.76,0.79,3,1,3,2,1,0,13.546724,16.181616,16.679368,1,18.391002,1.711634,0.000333,20.180527,20.651480
3,medconflict,2.625,5.37,0.44,4,1,2,1,0,0,21.164352,23.800127,24.299273,1,25.886782,1.587509,0.000264,27.810139,NaN
4,highconflict,1.750,7.32,0.62,5,1,2,1,1,1,28.801437,30.554489,31.059458,1,32.486818,1.427360,0.000523,34.558447,35.036377
5,lowconflict,2.625,11.71,0.45,6,1,2,3,1,1,35.558383,38.191502,38.691062,1,39.974525,1.283464,0.000334,42.191178,42.666810
6,lowconflict,1.750,11.46,0.22,7,1,1,2,0,1,43.191863,44.943835,45.442983,1,46.439034,0.996051,0.000946,48.955325,NaN
7,medconflict,5.250,14.75,0.74,8,1,3,3,1,1,49.945153,55.198869,55.703824,1,56.966082,1.262258,0.000361,59.202870,59.675642
8,medconflict,2.625,13.29,0.66,9,1,3,3,0,1,60.202773,62.819972,63.318612,0,64.613845,1.295232,0.000447,66.836814,NaN
9,medconflict,2.625,5.85,0.55,10,1,2,1,1,1,67.834346,70.473042,70.955503,0,72.237634,1.282132,0.000309,74.465987,NaN


,ConflictType,ITI,Reward,Risk,Trial,TimeType,RiskType,RewardType,Shock,LeftKey,FixOnset,RiskOnset,RewardOnset,ResponseType,ResponseOnset,ResponseTime,ResponseUncertainty,ResultOnset,ShockOnset
0,medconflict,1.750,7.32,0.19,1,1,1,1,0,0,0.014883,1.783063,2.282012,1,3.150720,0.868708,0.000299,5.783638,NaN
1,highconflict,1.750,13.90,0.85,2,1,3,3,1,0,6.784203,8.530830,9.046253,0,11.030596,1.984342,0.000413,12.536887,NaN
2,highconflict,2.625,9.76,0.79,3,1,3,2,1,0,13.528559,16.162472,16.661119,0,17.678622,1.017503,0.000347,20.160272,NaN
3,medconflict,2.625,5.37,0.44,4,1,2,1,1,0,21.160230,23.794132,24.292914,0,26.606225,2.313311,0.000312,27.794490,NaN
4,highconflict,1.750,7.32,0.62,5,1,2,1,0,1,28.795182,30.559318,31.063050,1,32.502184,1.439134,0.000265,34.557103,NaN
5,lowconflict,2.625,11.71,0.45,6,1,2,3,1,0,35.548546,38.175070,38.700115,1,39.518200,0.818085,0.000405,42.188947,42.657138
6,lowconflict,1.750,11.46,0.22,7,1,1,2,1,0,43.172045,44.939513,45.438235,1,46.333962,0.895727,0.000301,48.938410,49.412032
7,medconflict,5.250,14.75,0.74,8,1,3,3,1,0,49.939068,55.192935,55.708614,1,57.182078,1.473464,0.000417,59.193550,59.667612
8,medconflict,2.625,13.29,0.66,9,1,3,3,0,0,60.194140,62.829965,63.345806,0,64.357685,1.011879,0.000281,66.833496,NaN
9,medconflict,2.625,5.85,0.55,10,1,2,1,0,0,67.816576,70.467127,70.966220,0,71.949490,0.983270,0.000226,74.451278,NaN


newARC_009 does not have run 2


...

,ConflictType,ITI,Reward,Risk,Trial,TimeType,RiskType,RewardType,Shock,LeftKey,FixOnset,RiskOnset,RewardOnset,ResponseType,ResponseOnset,ResponseTime,ResponseUncertainty,ResultOnset,ShockOnset
0,medconflict,1.750,7.32,0.19,1,1,1,1,0,0,0.162354,1.855066,2.346586,1,3.367680,1.021093,0.001140,5.787719,NaN
1,highconflict,1.750,13.90,0.85,2,1,3,3,1,0,6.786086,8.536940,9.069101,1,10.006699,0.937598,0.000356,12.555718,13.019858
2,highconflict,2.625,9.76,0.79,3,1,3,2,1,0,13.540174,16.194281,16.684090,1,17.702642,1.018551,0.000402,20.183088,20.648688
3,medconflict,2.625,5.37,0.44,4,1,2,1,1,0,21.166379,23.819489,24.318945,0,25.550747,1.231802,0.000549,27.806003,NaN
4,highconflict,1.750,7.32,0.62,5,1,2,1,0,1,28.807256,30.577053,31.076509,1,31.942265,0.865756,0.000280,34.563544,NaN
5,lowconflict,2.625,11.71,0.45,6,1,2,3,1,0,35.548048,38.197132,38.695778,1,39.350390,0.654611,0.000394,42.195075,42.664670
6,lowconflict,1.750,11.46,0.22,7,1,1,2,1,0,43.194855,44.962323,45.461068,1,46.158398,0.697330,0.000534,48.946484,49.421976
7,medconflict,5.250,14.75,0.74,8,1,3,3,1,0,49.947599,55.209990,55.708806,1,56.374873,0.666067,0.001200,59.208026,59.676174
8,medconflict,2.625,13.29,0.66,9,1,3,3,0,0,60.192246,62.846533,63.345983,1,63.949651,0.603668,0.000306,66.839636,NaN
9,medconflict,2.625,5.85,0.55,10,1,2,1,0,0,67.834290,70.457458,70.971088,1,72.933876,1.962789,0.000420,74.475064,NaN


,ConflictType,ITI,Reward,Risk,Trial,TimeType,RiskType,RewardType,Shock,LeftKey,FixOnset,RiskOnset,RewardOnset,ResponseType,ResponseOnset,ResponseTime,ResponseUncertainty,ResultOnset,ShockOnset
0,medconflict,1.750,7.32,0.19,1,1,1,1,0,0,0.027443,1.778111,2.276972,0,3.463854,1.186882,0.000426,5.797773,NaN
1,highconflict,1.750,13.90,0.85,2,1,3,3,1,0,6.798032,8.549452,9.048224,1,9.839521,0.791297,0.000228,12.547611,13.010343
2,highconflict,2.625,9.76,0.79,3,1,3,2,1,0,13.548080,16.174419,16.673248,0,17.511562,0.838314,0.000186,20.193465,NaN
3,medconflict,2.625,5.37,0.44,4,1,2,1,1,0,21.173013,23.798713,24.297784,0,25.399755,1.101971,0.000339,27.818493,NaN
4,highconflict,1.750,7.32,0.62,5,1,2,1,0,1,28.797791,30.548305,31.068992,0,32.343937,1.274945,0.000530,34.568559,NaN
5,lowconflict,2.625,11.71,0.45,6,1,2,3,1,0,35.547794,38.195245,38.694117,1,39.703822,1.009705,0.000382,42.193321,42.656012
6,lowconflict,1.750,11.46,0.22,7,1,1,2,1,0,43.194315,44.945358,45.444125,1,46.199470,0.755345,0.000226,48.943418,49.412562
7,medconflict,5.250,14.75,0.74,8,1,3,3,1,0,49.944018,55.172866,55.694009,1,56.463892,0.769883,0.000499,59.214430,59.665617
8,medconflict,2.625,13.29,0.66,9,1,3,3,0,0,60.194074,62.820357,63.318567,1,64.063645,0.745078,0.000361,66.839454,NaN
9,medconflict,2.625,5.85,0.55,10,1,2,1,0,0,67.818619,70.444919,70.943648,0,71.855752,0.912105,0.000413,74.464382,NaN


,ConflictType,ITI,Reward,Risk,Trial,TimeType,RiskType,RewardType,Shock,LeftKey,FixOnset,RiskOnset,RewardOnset,ResponseType,ResponseOnset,ResponseTime,ResponseUncertainty,ResultOnset,ShockOnset
0,medconflict,1.750,7.32,0.19,1,1,1,1,0,0,0.156469,1.841484,2.324192,1,5.175180,2.850988,0.000299,5.793884,NaN
1,highconflict,1.750,13.90,0.85,2,1,3,3,1,0,6.784002,8.535473,9.050243,0,9.879093,0.828851,0.000295,12.550088,NaN
2,highconflict,2.625,9.76,0.79,3,1,3,2,1,0,13.534528,16.173028,16.671451,0,17.679605,1.008154,0.000564,20.181035,NaN
3,medconflict,2.625,5.37,0.44,4,1,2,1,1,0,21.180915,23.798954,24.313633,1,25.839031,1.525397,0.000279,27.817294,28.282483
4,highconflict,1.750,7.32,0.62,5,1,2,1,0,1,28.801904,30.555583,31.062081,0,32.838963,1.776882,0.000261,34.561228,NaN
5,lowconflict,2.625,11.71,0.45,6,1,2,3,1,0,35.561214,38.180219,38.696265,1,39.854914,1.158648,0.000297,42.200207,42.668489
6,lowconflict,1.750,11.46,0.22,7,1,1,2,1,0,43.192958,44.944352,45.453885,1,46.991179,1.537294,0.000551,48.941490,49.421486
7,medconflict,5.250,14.75,0.74,8,1,3,3,1,0,49.942239,55.199327,55.706953,0,56.686666,0.979713,0.000276,59.205968,NaN
8,medconflict,2.625,13.29,0.66,9,1,3,3,0,0,60.205906,62.825288,63.338660,0,64.350931,1.012271,0.000368,66.837875,NaN
9,medconflict,2.625,5.85,0.55,10,1,2,1,0,0,67.829070,70.456645,70.970361,0,72.102657,1.132296,0.000271,74.469875,NaN


,ConflictType,ITI,Reward,Risk,Trial,TimeType,RiskType,RewardType,Shock,LeftKey,FixOnset,RiskOnset,RewardOnset,ResponseType,ResponseOnset,ResponseTime,ResponseUncertainty,ResultOnset,ShockOnset
0,medconflict,1.750,7.32,0.19,1,1,1,1,0,0,0.156472,1.852902,2.350183,1,3.343061,0.992877,0.000394,5.786279,NaN
1,highconflict,1.750,13.90,0.85,2,1,3,3,1,0,6.786120,8.555893,9.071920,1,9.839368,0.767448,0.000787,12.549520,13.018247
2,highconflict,2.625,9.76,0.79,3,1,3,2,1,0,13.543390,16.180919,16.697072,1,17.590592,0.893520,0.000474,20.167464,20.646409
3,medconflict,2.625,5.37,0.44,4,1,2,1,1,0,21.168554,23.798704,24.305317,0,25.190049,0.884731,0.000325,27.812458,NaN
4,highconflict,1.750,7.32,0.62,5,1,2,1,0,1,28.795580,30.563382,31.079555,1,32.301959,1.222405,0.000452,34.566521,NaN
5,lowconflict,2.625,11.71,0.45,6,1,2,3,1,0,35.551017,38.194463,38.704537,1,39.501874,0.797337,0.000440,42.192205,42.662144
6,lowconflict,1.750,11.46,0.22,7,1,1,2,1,0,43.176069,44.959481,45.458157,1,46.189505,0.731348,0.000339,48.949090,49.417414
7,medconflict,5.250,14.75,0.74,8,1,3,3,1,0,49.940435,55.190345,55.706394,1,56.389281,0.682887,0.000442,59.204891,59.672524
8,medconflict,2.625,13.29,0.66,9,1,3,3,0,0,60.194776,62.838464,63.337253,1,63.924722,0.587469,0.000282,66.836211,NaN
9,medconflict,2.625,5.85,0.55,10,1,2,1,0,0,67.819788,70.474871,70.973335,0,72.084568,1.111232,0.000471,74.467814,NaN


newARC_020 does not have run 1
